In [1]:
%%capture
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install faiss-gpu
!pip install einops
!pip install --upgrade --quiet  langchain-community chromadb bs4 qdrant-client
!pip install langchainhub
!pip install sentence-transformers
!pip install tqdm
!pip install rouge_score nltk bert_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/DATASCI266/project')

In [4]:
from summary_utils import SummaryEvaluator, DatasetManager, SummaryModel

In [5]:
import os
import re
import torch
from datasets import load_dataset
from tqdm import tqdm
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain import PromptTemplate, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer , AutoModelForCausalLM
from transformers import pipeline, BitsAndBytesConfig

In [6]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

In [7]:
summary_evaluator = SummaryEvaluator()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Load the XSum dataset
data_manager = DatasetManager(sample_size=200)
dataset = data_manager.load_sampled_dataset(dataset_label="test")

# Initialize the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
%%capture

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         llm_int4_enable_fp32_cpu_offload=True)


llm_mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    torch_dtype=torch.float32,
    device_map='auto',
    quantization_config=quantization_config
)

llm_mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

mistral_pipe = pipeline(
    "text-generation",
    model=llm_mistral_model,
    tokenizer=llm_mistral_tokenizer,
    max_length=1000,
    temperature=0.6,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.2
)
mistral_pipe.model.config.pad_token_id = mistral_pipe.model.config.eos_token_id

mistral_llm_lc = HuggingFacePipeline(pipeline=mistral_pipe)

Unused kwargs: ['llm_int4_enable_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [10]:
t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

t5_pipe = pipeline(
    "summarization",
    model=t5model,
    tokenizer=t5tokenizer,
    max_length=1000,
    truncation=True,
    device=0
)

t5_llm_lc = HuggingFacePipeline(pipeline=t5_pipe)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
class RAG_Summarization_Model:
  def __init__(self, chunk_size=200, chunk_overlap=50, model=None, model_name=None):
    self.vector_store = None
    self.rag_chain = None
    self.chunk_size = chunk_size
    self.chunk_overlap = chunk_overlap
    self.model = model
    self.model_name = model_name

  def fit(self, dataset):
    # Function to split documents and include metadata
    def create_documents_with_metadata(dataset, chunk_size=200, chunk_overlap=50):
      # Initialize the text splitter
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=chunk_size,  # Adjust based on your needs
          chunk_overlap=chunk_overlap  # Adjust based on your needs
      )
      documents = []
      for example in dataset:
          text = example['document']
          # Split the text into chunks while including metadata
          chunks = text_splitter.split_text(text)
          for chunk in chunks:
              documents.append(Document(page_content=chunk, metadata={"id": example['id']}))
      return documents
    # Create documents with metadata
    documents = create_documents_with_metadata(dataset, self.chunk_size, self.chunk_overlap)
    # Create a FAISS vector store
    self.vector_store = FAISS.from_documents(documents, embeddings)

    if self.model_name == 'mistral':
      rag_template = """[INST] Give me a short one-sentence summarization with the following context: {context}[/INST]"""
    elif self.model_name == 't5-base':
      rag_template = """summarize: {context}"""
    else:
      raise ValueError("Invalid model type")

    rag_prompt = ChatPromptTemplate.from_template(rag_template)

    self.rag_chain = (
        {"context": lambda inputs: self.format_docs(self.query_vector_store(inputs['query'], inputs['id'], k=inputs['k'], fetch_k=inputs['fetch_k'])),
        "question": RunnablePassthrough()}
        | rag_prompt
        | self.model
    )

  def query_vector_store(self, query_text, summary_id, k=3, fetch_k=20):
    # Embed the query text
    query_vector = embeddings.embed_query(query_text)
    # Search the vector store
    docs = self.vector_store.similarity_search_by_vector(query_vector, filter=dict(id=summary_id), k=k, fetch_k=fetch_k)
    return docs

  def format_docs(self, docs):
      return "\n\n".join(doc.page_content for doc in docs)

  def predict(self, dataset, k, fetch_k, query):
    results = []
    i = 1
    for example in tqdm(dataset):
        print(i)
        text = example['document']
        sentences = re.split(r'[.?!]\s+', text)
        if query == 'first_sentence':
          example_query = sentences[0]
        elif query == 'last_sentence':
          example_query = sentences[-1]
        elif query == 'first_last_sentence':
          example_query = sentences[0] + '. ' + sentences[-1]
        else:
          raise ValueError("Invalid query type")

        summary_w_prompt = self.rag_chain.invoke({"query": example_query, "id": example['id'], "k":k, 'fetch_k':fetch_k})

        if self.model_name == 'mistral':
          summary = summary_w_prompt.split("[/INST] ",1)[1]
        elif self.model_name == 't5-base':
          summary = summary_w_prompt
        else:
          raise ValueError("Invalid model type")
        results.append(summary)
        i += 1
    return results

In [13]:
dataset_analyze = dataset.take(5)

In [14]:
references = [example['summary'] for example in dataset_analyze]

In [17]:
for reference in references:
  print('Reference: ' + reference)

Reference: A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
Reference: A Tudor manor house has reopened following a £2.2m makeover.
Reference: Walt Disney World has unveiled a lighthouse memorial for a young boy who was killed by an alligator while on holiday at the Florida theme park.
Reference: The agriculture minister is to return to China next month as attempts to access markets there continue.
Reference: Blackpool have signed Bournemouth goalkeeper Ryan Allsop and midfielder Callum Cooke from Middlesbrough on season-long loans.


In [18]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=50, model=mistral_llm_lc, model_name='mistral')
rag_model.fit(dataset_analyze)
mistral_rag_10_first = rag_model.predict(dataset_analyze, k=10, fetch_k=50, query='first_sentence')
mistral_rag_10_last = rag_model.predict(dataset_analyze, k=10, fetch_k=50, query='last_sentence')
mistral_rag_10_both = rag_model.predict(dataset_analyze, k=10, fetch_k=50, query='first_last_sentence')

  0%|          | 0/5 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


 20%|██        | 1/5 [00:07<00:29,  7.47s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


 40%|████      | 2/5 [00:14<00:20,  6.94s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 60%|██████    | 3/5 [00:29<00:19,  9.54s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


4


 80%|████████  | 4/5 [00:42<00:10, 10.85s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


5


100%|██████████| 5/5 [00:45<00:00,  9.08s/it]


In [19]:
t5_rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=50, model=t5_llm_lc, model_name='t5-base')
t5_rag_model.fit(dataset_analyze)
t5_rag_10_first = t5_rag_model.predict(dataset_analyze, k=10, fetch_k=50, query='first_sentence')
t5_rag_10_last = t5_rag_model.predict(dataset_analyze, k=10, fetch_k=50, query='last_sentence')
t5_rag_10_both = t5_rag_model.predict(dataset_analyze, k=10, fetch_k=50, query='first_last_sentence')

  0%|          | 0/5 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 1000, but your input_length is only 330. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=165)


1


 20%|██        | 1/5 [00:36<02:26, 36.67s/it]Your max_length is set to 1000, but your input_length is only 222. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=111)


2


 40%|████      | 2/5 [00:58<01:24, 28.18s/it]Your max_length is set to 1000, but your input_length is only 365. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=182)


...

 60%|██████    | 3/5 [01:13<00:50, 25.06s/it]Your max_length is set to 1000, but your input_length is only 219. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=109)


4


 80%|████████  | 4/5 [01:31<00:22, 22.14s/it]Your max_length is set to 1000, but your input_length is only 87. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


5


100%|██████████| 5/5 [01:44<00:00, 20.91s/it]


In [36]:
for i in range(5):
  print('EXAMPLE: ' + str(i+1))
  print('Reference: ' + references[i])
  print('Mistral: ' + mistral_rag_10_first[i])
  print('T5: ' + t5_rag_10_first[i])
  print('\n')

EXAMPLE: 1
Reference: A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
Mistral: In April 2013, Sarah Johnson and her friends were involved in a fatal accident on the M62 while traveling to a hen party, resulting in the death of her friend Bethany Jones and injuries to several others including herself.
T5: 21 women were heading to Liverpool when their minibus was hit by a lorry . their friend, 18-year-old Bethany Jones, was killed in the crash . minibus driver james johnson was jailed for more than six years for causing her death .


EXAMPLE: 2
Reference: A Tudor manor house has reopened following a £2.2m makeover.
Mistral: The renovated Bramall Hall in Stockport, which features restored stained glass windows and ceilings as well as access to previously unseen areas, has sold out all 1,400 tickets during its opening weekend.
T5: bramall hall in stockport, gm, is the "jewel in the crown" of the town's herita

In [37]:
for i in range(5):
  print('EXAMPLE: ' + str(i+1))
  print('Reference: ' + references[i])
  print('Mistral: ' + mistral_rag_10_last[i])
  print('T5: ' + t5_rag_10_last[i])
  print('\n')

EXAMPLE: 1
Reference: A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
Mistral: Ms. Johnson realized how lucky she was after surviving a serious accident and decided to use her experience to help others through a charity called Day One that offers benefits, legal advice, and peer support to those affected by major trauma.
T5: Sarah Johnson was one of 21 women heading to Liverpool when their minibus was hit by a lorry . her friend Bethany Jones, 18, was killed while she and several others were badly hurt . she broke her shoulder, back and pelvis and spent time in intensive care .


EXAMPLE: 2
Reference: A Tudor manor house has reopened following a £2.2m makeover.
Mistral: Renovations at Bramall Hall in Stockport were completed with a £1.6m grant from the Heritage Lottery Fund and £400,000 in council funding, resulting in restored stained glass windows and ceilings, as well as new visitor access to the dinin

In [38]:
for i in range(5):
  print('EXAMPLE: ' + str(i+1))
  print('Reference: ' + references[i])
  print('Mistral: ' + mistral_rag_10_both[i])
  print('T5: ' + t5_rag_10_both[i])
  print('\n')

EXAMPLE: 1
Reference: A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
Mistral: In an interview, Sarah Johnson shared her experience of surviving a fatal car accident that killed her friend Bethany Jones during a hen party trip. She expressed gratitude for being alive but also mourned the loss of her friend. While recovering from injuries at Leeds General Infirmary, Ms. Johnson discovered a charity called Day One, which provided support to victims of major trauma like herself.
T5: 21 women were heading to Liverpool when their minibus was hit by a lorry . their friend, 18-year-old Bethany Jones, was killed in the crash . the minibus driver was jailed for more than six years for causing her friend's death .


EXAMPLE: 2
Reference: A Tudor manor house has reopened following a £2.2m makeover.
Mistral: The renovated Bramall Hall in Stockport, which received funding from the Heritage Lottery and council, has sol

In [39]:
def analyze_results(example, reference, model, k, query):
  print('Reference: ' + reference)
  text = example['document']
  sentences = re.split(r'[.?!]\s+', text)
  if query == 'first_sentence':
    example_query = sentences[0]
  elif query == 'last_sentence':
    example_query = sentences[-1]
  elif query == 'first_last_sentence':
    example_query = sentences[0] + '. ' + sentences[-1]

  docs = model.query_vector_store(example_query, example['id'], k=k, fetch_k=50)
  for doc in docs:
    print('Document: ' + doc.page_content)

In [41]:
print('EXAMPLE 1')
print('first_sentence:')
analyze_results(dataset_analyze[0], references[0], rag_model, 5, 'first_sentence')
print('\n')
print('last_sentence:')
analyze_results(dataset_analyze[0], references[0], rag_model, 5, 'last_sentence')
print('\n')
print('first_last_sentence:')
analyze_results(dataset_analyze[0], references[0], rag_model, 5, 'first_last_sentence')
print('\n')

EXAMPLE 1
first_sentence:
Reference: A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
Document: Sarah Johnson was one of 21 women heading to Liverpool when their minibus was hit by a lorry on the M62.
Her friend Bethany Jones, 18, was killed while Ms Johnson and several others were badly hurt.
Document: Minibus driver James Johnson was jailed for more than six years for causing Bethany's death, in April 2013.
Document: Ms Johnson was taken to Leeds General Infirmary where she, along with Bethany's sister Amy Firth, underwent major surgery and spent time in intensive care.
Document: Ms Johnson said the crash had made her realise how lucky she had been.
"Beth can't complain, she's not here," she added.  "We just have to be grateful for what we've got."
Document: Speaking publicly for the first time about the crash, Ms Johnson described how everyone was "excited and giddy" for the hen party.


last_sentence:


In [42]:
print('EXAMPLE 2')
print('first_sentence:')
analyze_results(dataset_analyze[1], references[1], rag_model, 5, 'first_sentence')
print('\n')
print('last_sentence:')
analyze_results(dataset_analyze[1], references[1], rag_model, 5, 'last_sentence')
print('\n')
print('first_last_sentence:')
analyze_results(dataset_analyze[1], references[1], rag_model, 5, 'first_last_sentence')
print('\n')

EXAMPLE 2
first_sentence:
Reference: A Tudor manor house has reopened following a £2.2m makeover.
Document: A total of 1,400 tickets have sold out for the opening weekend at Bramall Hall in Stockport, Greater Manchester after renovation work began in 2014.
Document: Councillor Kate Butler, from Stockport Council, called it the "jewel in the crown" of the town's heritage.
Document: Stained glass windows and ceilings have been restored, while the public will be able to visit the dining room and butler's pantry for the first time.
Document: In 1936, the timber-framed hall and its surrounding parkland was handed over to the local council.
Document: Since then the estate has been under the ownership of just three families: the Davenports, De Bromales and Nevills.


last_sentence:
Reference: A Tudor manor house has reopened following a £2.2m makeover.
Document: The transformation followed a £1.6m grant from the Heritage Lottery Fund and £400,000 in council funding.
Document: In 1936, the tim

In [43]:
print('EXAMPLE 3')
print('first_sentence:')
analyze_results(dataset_analyze[2], references[2], rag_model, 5, 'first_sentence')
print('\n')
print('last_sentence:')
analyze_results(dataset_analyze[2], references[2], rag_model, 5, 'last_sentence')
print('\n')
print('first_last_sentence:')
analyze_results(dataset_analyze[2], references[2], rag_model, 5, 'first_last_sentence')
print('\n')

EXAMPLE 3
first_sentence:
Reference: Walt Disney World has unveiled a lighthouse memorial for a young boy who was killed by an alligator while on holiday at the Florida theme park.
Document: Two-year-old Lane Thomas Graves had been playing in the sand near the resort's Seven Seas Lagoon when he was dragged underwater by the creature.
Document: "He was in the water not more than ankle deep," the Florida Fish and Wildlife Conservation Commission said in a report, describing how the boy had been gathering water for a sandcastle.
Document: His father, Matt Graves, jumped in the water to try to pry open the creature's mouth, but "the alligator thrashed and broke Matt's grasp and went under the water," according to the report.
Document: After the death, Disney was criticised for not having posted signs warning of the danger along the man-made lagoon, which borders Magic Kingdom.
Document: A Disney spokesperson said they hoped the monument would spread awareness for the Lane Thomas Foundation

In [44]:
print('EXAMPLE 4')
print('first_sentence:')
analyze_results(dataset_analyze[3], references[3], rag_model, 5, 'first_sentence')
print('\n')
print('last_sentence:')
analyze_results(dataset_analyze[3], references[3], rag_model, 5, 'last_sentence')
print('\n')
print('first_last_sentence:')
analyze_results(dataset_analyze[3], references[3], rag_model, 5, 'first_last_sentence')
print('\n')

EXAMPLE 4
first_sentence:
Reference: The agriculture minister is to return to China next month as attempts to access markets there continue.
Document: Michelle O'Neill hosted a departmental breakfast at the Balmoral Show on Thursday morning.
In attendance was the Chinese consul general Madam Wang Shuying.
Document: It is the second day of the Balmoral Show, the biggest showcase for Northern Ireland's farming and food industries.
Document: Ms O'Neill said she was also hoping to agree access to Chinese markets for beef and chicken.
US consul general Greg Burton was also at the event.
Document: Ms O'Neill said she was also working with Irish agriculture minister Simon Coveney to resolve labelling issues that have hit Northern Ireland lamb sales to meat plants in the Republic of Ireland.
Document: She told guests that the quality of Northern Ireland food was "something that carries our reputation around the world."


last_sentence:
Reference: The agriculture minister is to return to China 

In [46]:
print('EXAMPLE 5')
print('first_sentence:')
analyze_results(dataset_analyze[4], references[4], rag_model, 5, 'first_sentence')
print('\n')
print('last_sentence:')
analyze_results(dataset_analyze[4], references[4], rag_model, 5, 'last_sentence')
print('\n')
print('first_last_sentence:')
analyze_results(dataset_analyze[4], references[4], rag_model, 5, 'first_last_sentence')
print('\n')

EXAMPLE 5
first_sentence:
Reference: Blackpool have signed Bournemouth goalkeeper Ryan Allsop and midfielder Callum Cooke from Middlesbrough on season-long loans.
Document: Allsop, 25, made his Premier League debut in November 2015 and has spent time on loan at Coventry and Wycombe.
Document: Cooke, 20, won the European Championship with England Under-17s in 2014 and scored four goals while on loan at Crewe last season.
League One Blackpool have now signed nine players this summer.
Document: Find all the latest football transfers on our dedicated page.


last_sentence:
Reference: Blackpool have signed Bournemouth goalkeeper Ryan Allsop and midfielder Callum Cooke from Middlesbrough on season-long loans.
Document: Find all the latest football transfers on our dedicated page.
Document: Allsop, 25, made his Premier League debut in November 2015 and has spent time on loan at Coventry and Wycombe.
Document: Cooke, 20, won the European Championship with England Under-17s in 2014 and scored f